In [3]:
!wget -q https://git.io/JGc31 -O ucf101_top5.tar.gz
!tar xf ucf101_top5.tar.gz

tar: This does not look like a tar archive

gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [4]:
!pip install -q git+https://github.com/tensorflow/docs

In [5]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.applications import InceptionResNetV2, ResNet50, InceptionV3, DenseNet121, Xception
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

2023-06-08 04:05:45.206630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 04:05:50.332414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
MAX_SEQ_LENGTH = 8  #20
NUM_FEATURES = 1024
IMG_SIZE = 128

EPOCHS = 10

In [7]:
def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)
    print(preprocessed.shape)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

2023-06-08 04:06:06.065612: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7962 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB MIG 2g.10gb, pci bus id: 0000:21:00.0, compute capability: 8.0


(None, 128, 128, 3)


In [8]:
X_data = np.load('../data/X_data.npy', allow_pickle=True)
y_data = np.load('../data/y_data.npy', allow_pickle=True)
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(y_data), mask_token=None
)
print(label_processor.get_vocabulary())
print(label_processor(['hiatal_dissec','eso_mob']))
print(X_data.shape)

['eso_mob', 'exposure', 'fundus_mob', 'hiatal_dissec', 'hiatal_repair', 'oob', 'peg_placement', 'sac_excision', 'wrap']
tf.Tensor([3 0], shape=(2,), dtype=int64)
(4778, 8, 128, 128, 3)


In [ ]:
def prepare_all_videos(data, labels):
    num_samples = data.shape[0]
    labels = label_processor(labels).numpy()

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )
    # For each video.
    for i in range(num_samples):
        frames = data[i]
        # Initialize placeholder to store the features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )
        # Extract features from the frames of the current video.
        for j in range(8):
            temp_frame_features[0, j, :] = feature_extractor.predict(
                frames[None,j]
            )
        frame_features[i,] = temp_frame_features.squeeze()
        print(i)

    return frame_features, labels

frame_features, labels = prepare_all_videos(X_data, y_data)

1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 24ms/step
1
1/1 [==============================] - 0s 25ms/step
2
1/1 [==============================] - 0s 25ms/step
3
1/1 [==============================] - 0s 25ms/step
4
1/1 [==============================] - 0s 24ms/step
5
1/1 [==============================] - 0s 24ms/step
6
1/1 [==============================] - 0s 27ms/step
7
1/1 [==============================] - 0s 25ms/step
8
1/1 [==============================] - 0s 26ms/step
9
1/1 [==============================] - 0s 25ms/step
10
1/1 [==============================] - 0s 25ms/step
11
1/1 [==============================] - 0s 25ms/step
12
1/1 [==============================] - 0s 24ms/step
13
1/1 [==============================] - 0s 25ms/step
14
1/1 [==============================] - 0s 28ms/step
15
1/1 [==============================] - 0s 26ms/step
16
1/1 [==============================] - 0s 25ms/step
17
1/1 [===============

In [21]:
print(frame_features.shape)
np.save('frame_features_Densenet.npy',frame_features)
np.save('labels.npy',labels)

NameError: name 'frame_features' is not defined

In [9]:
X = np.load('frame_features_Densenet.npy', allow_pickle=True)
y = np.load('labels.npy',allow_pickle=True)
print(X.shape)
print(y.shape)

(4778, 8, 1024)
(4778,)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(3822, 8, 1024)
(956, 8, 1024)
(3822,)


In [11]:
unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  0 646]
 [  1 232]
 [  2 148]
 [  3 536]
 [  4 723]
 [  5 315]
 [  6 119]
 [  7 273]
 [  8 830]]


In [12]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [13]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [14]:
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    classes = len(label_processor.get_vocabulary())

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = "../source/vit_checkpoint"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    model = get_compiled_model()
    history = model.fit(
        X_train,
        y_train,
        validation_split=0.15,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    model.load_weights(filepath)
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

In [15]:
trained_model = run_experiment()

Epoch 1/10


2023-06-08 04:06:18.574056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-08 04:06:18.750053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8801
2023-06-08 04:06:18.827675: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f79a5d41e50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-08 04:06:18.827713: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB MIG 2g.10gb, Compute Capability 8.0
2023-06-08 04:06:18.831883: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-08 04:06:18.852412: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:530] Can't find libdevice directory ${CUDA_DIR}/nvvm/l

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_18' defined at (most recent call last):
    File "/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/tornado/platform/a